In [1]:
import pandas as pd
from pyomo.environ import *
from datetime import datetime, timedelta, time


In [2]:
df_shipments = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_shipments')
df_schedule = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_schedule')
df_capacity = pd.read_excel("01-OR-RS-homework-data-template.xlsx", sheet_name='df_capacity')

In [ ]:
model = ConcreteModel()

In [ ]:
# Sets
model.Shipments = Set(initialize=df_shipments['shipment_id'].unique())
model.Lanes = Set(initialize=df_schedule.index)

In [ ]:
# Define parameters
model.Weight = Param(model.Shipments, initialize=df_shipments.set_index('shipment_id')['weight_kg'].to_dict())
model.DueTime = Param(model.Shipments, initialize=df_shipments.set_index('shipment_id')['due_time_local'].to_dict())
model.OriginRegion = Param(model.Shipments, initialize=df_shipments.set_index('shipment_id')['origin_region'].to_dict())
model.DestinationRegion = Param(model.Shipments, initialize=df_shipments.set_index('shipment_id')['destination_region'].to_dict())
model.ReadyTime = Param(model.Shipments, initialize=df_shipments.set_index('shipment_id')['ready_time_local'].to_dict())

model.CarrierID = Param(model.Lanes, initialize=df_schedule['carrier_id'].to_dict())
model.LaneRateType = Param(model.Lanes, initialize=df_schedule['rate_type'].to_dict())
model.LaneRate = Param(model.Lanes, initialize=df_schedule['rate_type'].to_dict(), default=0)
model.LaneRateBreak = Param(model.Lanes, initialize=df_schedule['rate_break'].to_dict(), default='')

model.LaneOrigin = Param(model.Lanes, initialize=df_schedule['origin_region'].to_dict())
model.LaneDestination = Param(model.Lanes, initialize=df_schedule['destination_region'].to_dict())


In [ ]:

import ast

# Function to convert string representation of list of tuples to actual list of tuples
def parse_departure_times(dep_times_str):
    return ast.literal_eval(dep_times_str)

def calculate_arrival_times(schedule):
    arrival_times = {}
    for lane in schedule.index:
        dep_times = parse_departure_times(schedule.at[lane, 'scheduled_depart_weekday_time_local'])
        travel_time = schedule.at[lane, 'travel_hrs_with_timezone_offset']
        arrival_times[lane] = [(day, (datetime.combine(datetime.today(), datetime.strptime(dep_time, '%H:%M').time()) + timedelta(hours=travel_time)).time()) for day, dep_time in dep_times]
    return arrival_times

arrival_times = calculate_arrival_times(df_schedule)

In [ ]:
# Objective: Minimize cost
def cost_rule(model):
    cost = 0
    for s in model.Shipments:
        for l in model.Lanes:
            if lane_rate_type[l] == 'FTL':
                cost += model.x[s, l] * lane_rate_break[l] * (shipment_weight[s] / df_capacity.loc[0, 'max_weight_kgs'])
            else:
                rate_break = eval(lane_rate_break[l])
                cost_per_kg = next(rate for weight, rate in sorted(rate_break.items()) if shipment_weight[s] >= weight)
                cost += model.x[s, l] * shipment_weight[s] * cost_per_kg
    return cost
model.cost = Objective(rule=cost_rule, sense=minimize)